# PROJETO EDA - ANÁLISE DE SENTIMENTOS CYBERPUNK 2077

## Disciplina: IMD1151 - Ciência de Dados

### PROBLEMAS:

- Como a percepção dos jogadores sobre Cyberpunk 2077 evoluiu ao longo
do tempo (desde o lançamento problemático em 2020 até 2025)?
- Quais
diferenças existem entre as comunidades de língua inglesa e portuguesa
em termos de sentimento e aspectos mais valorizados/criticados?

**Esse notebook coleta os dados via API da steam**

In [ ]:
import requests
import pandas as pd
import time

In [ ]:
# O ID do Cyberpunk 2077 na Steam
app_id = 1091500

params = {
    'json': 1,
    'filter': 'all',
    'language': 'brazilian',
    'review_type': 'all',
    'purchase_type': 'all',
    'day_range': '99999'
}

In [ ]:
all_reviews = []
cursor = '*'

max_reviews_to_fetch = 100000



In [ ]:
def coletar_reviews_steam(idioma, app_id=1091500, max_reviews=50000):
    """
    Coleta avaliações da Steam para um determinado idioma.
    Retorna um DataFrame do Pandas com os resultados.
    """
    print(f"--- Iniciando a coleta para o idioma: {idioma.upper()} ---")

    params = {
        'json': 1,
        'filter': 'all',
        'language': idioma,
        'review_type': 'all',
        'purchase_type': 'all',
        'day_range': '99999'
    }

    headers = {
        'User-Agent': 'ProjetoAcademicoCienciaDeDados-AnaliseCyberpunk/1.0 (lf06092004@gmail.com)'
    }

    all_reviews = []
    cursor = '*'

    while len(all_reviews) < max_reviews:
        params['cursor'] = cursor
        try:
            response = requests.get(f"https://store.steampowered.com/appreviews/{app_id}", params=params, headers=headers)
            response.raise_for_status()
            data = response.json()

            if data.get('success') != 1:
                print(f"API retornou falha para o idioma {idioma}. Interrompendo.")
                break

            reviews_on_page = data.get('reviews', [])
            if not reviews_on_page:
                print(f"Não há mais avaliações para o idioma {idioma}. Fim da coleta.")
                break

            all_reviews.extend(reviews_on_page)

            next_cursor = data.get('cursor')
            if not next_cursor or next_cursor == cursor:
                print(f"Cursor não atualizou para o idioma {idioma}. Fim da coleta.")
                break

            cursor = next_cursor
            print(f"Idioma: {idioma.upper()} | Coletadas {len(all_reviews)} de {max_reviews}...")
            time.sleep(1)

        except Exception as e:
            print(f"Ocorreu um erro na coleta para {idioma}: {e}")
            break

    print(f"--- Coleta finalizada para {idioma.upper()}. Total: {len(all_reviews)} avaliações. ---\n")
    return pd.DataFrame(all_reviews)

In [ ]:
df_br = coletar_reviews_steam(idioma='brazilian', max_reviews=50000)

--- Iniciando a coleta para o idioma: BRAZILIAN ---
Idioma: BRAZILIAN | Coletadas 20 de 50000...
Idioma: BRAZILIAN | Coletadas 40 de 50000...
Idioma: BRAZILIAN | Coletadas 60 de 50000...
Idioma: BRAZILIAN | Coletadas 80 de 50000...
Idioma: BRAZILIAN | Coletadas 100 de 50000...
Idioma: BRAZILIAN | Coletadas 120 de 50000...
Idioma: BRAZILIAN | Coletadas 140 de 50000...
Idioma: BRAZILIAN | Coletadas 160 de 50000...
Idioma: BRAZILIAN | Coletadas 180 de 50000...
Idioma: BRAZILIAN | Coletadas 200 de 50000...
Idioma: BRAZILIAN | Coletadas 220 de 50000...
Idioma: BRAZILIAN | Coletadas 240 de 50000...
Idioma: BRAZILIAN | Coletadas 260 de 50000...
Idioma: BRAZILIAN | Coletadas 280 de 50000...
Idioma: BRAZILIAN | Coletadas 300 de 50000...
Idioma: BRAZILIAN | Coletadas 320 de 50000...
Idioma: BRAZILIAN | Coletadas 340 de 50000...
Idioma: BRAZILIAN | Coletadas 360 de 50000...
Idioma: BRAZILIAN | Coletadas 380 de 50000...
Idioma: BRAZILIAN | Coletadas 400 de 50000...
Idioma: BRAZILIAN | Coletadas 42

In [ ]:
df_en = coletar_reviews_steam(idioma='english', max_reviews=50000)

--- Iniciando a coleta para o idioma: ENGLISH ---
Idioma: ENGLISH | Coletadas 20 de 50000...
Idioma: ENGLISH | Coletadas 40 de 50000...
Idioma: ENGLISH | Coletadas 60 de 50000...
Idioma: ENGLISH | Coletadas 80 de 50000...
Idioma: ENGLISH | Coletadas 100 de 50000...
Idioma: ENGLISH | Coletadas 120 de 50000...
Idioma: ENGLISH | Coletadas 140 de 50000...
Idioma: ENGLISH | Coletadas 160 de 50000...
Idioma: ENGLISH | Coletadas 180 de 50000...
Idioma: ENGLISH | Coletadas 200 de 50000...
Idioma: ENGLISH | Coletadas 220 de 50000...
Idioma: ENGLISH | Coletadas 240 de 50000...
Idioma: ENGLISH | Coletadas 260 de 50000...
Idioma: ENGLISH | Coletadas 280 de 50000...
Idioma: ENGLISH | Coletadas 300 de 50000...
Idioma: ENGLISH | Coletadas 320 de 50000...
Idioma: ENGLISH | Coletadas 340 de 50000...
Idioma: ENGLISH | Coletadas 360 de 50000...
Idioma: ENGLISH | Coletadas 380 de 50000...
Idioma: ENGLISH | Coletadas 400 de 50000...
Idioma: ENGLISH | Coletadas 420 de 50000...
Idioma: ENGLISH | Coletadas 44

In [ ]:
if not df_br.empty and not df_en.empty:
    print("Juntando os datasets...")
    df_completo = pd.concat([df_br, df_en], ignore_index=True)

    print(f"Tamanho total antes da limpeza de duplicatas: {len(df_completo)}")
    df_completo.drop_duplicates(subset=['recommendationid'], keep='first', inplace=True)

    output_filename = 'steam_reviews_cyberpunk_br_en.csv'
    df_completo.to_csv(output_filename, index=False)

    print(f"Arquivo final '{output_filename}' salvo com {len(df_completo)} avaliações únicas.")
else:
    print("Uma ou ambas as coletas falharam. Nenhum arquivo foi salvo.")

Juntando os datasets...
Tamanho total antes da limpeza de duplicatas: 84522
Arquivo final 'steam_reviews_cyberpunk_br_en.csv' salvo com 41427 avaliações únicas.
